In [9]:
#!pip3 install -I git+https://github.com/cloudera/cmlextensions.git
#!pip3 install -I ray
#!pip3 install -I modin[ray]
#!pip3 install transformers datasets scipy sklearn torch
#!pip3 install transformers[torch]
#!pip3 install accelerate -U
#!pip3 install ray[client]
#!pip3 install ray[tune]
#!pip3 install xgboost_ray

In [9]:
#!nvidia-smi

In [13]:
import cmlextensions.ray_cluster as rc
import cmlapi
import os
import json
from pprint import pprint
import ray

In [14]:
# Set the setup variables needed by CML APIv2
HOST = os.getenv("CDSW_API_URL").split(":")[0] + "://" + os.getenv("CDSW_DOMAIN")
USERNAME = os.getenv("CDSW_PROJECT_URL").split("/")[6]  # args.username  # "vdibia"
API_KEY = os.getenv("CDSW_APIV2_KEY")
PROJECT_NAME = os.getenv("CDSW_PROJECT")
PROJECT_ID=os.getenv("CDSW_PROJECT_ID")

cml = cmlapi.default_client(url=HOST,cml_api_key=API_KEY)

def set_environ(Cml,Item,Value):
    Project=Cml.get_project(os.getenv("CDSW_PROJECT_ID"))
    if Project.environment=='':
        Project_Environment={}
    else:
        Project_Environment=json.loads(Project.environment)
    Project_Environment[Item]=Value
    Project.environment=json.dumps(Project_Environment)
    Cml.update_project(Project,project_id=os.getenv("CDSW_PROJECT_ID"))

def get_environ(Cml,Item):
    Project=Cml.get_project(os.getenv("CDSW_PROJECT_ID"))
    Project_Environment=json.loads(Project.environment)
    return Project_Environment[Item]

In [15]:
#ray.shutdown()
#cluster.terminate()

In [16]:
cluster = rc.RayCluster( 
                        num_workers=2,
                         worker_cpu=2, worker_memory=4, worker_nvidia_gpu=0, 
                         head_cpu=2, head_memory=4, head_nvidia_gpu=0                       
                       )
cluster.init()
set_environ(cml,"RAY_ADDRESS",cluster.get_client_url())

Starting ray head...
Starting 2 ray workers...

--------------------
Ray cluster started
--------------------

The Ray dashboard is running at
https://1uj5mbn3865o6uoq.ml-b74f8940-b97.go01-dem.ylcu-atmi.cloudera.site/

To connect to this Ray cluster from this CML Session,
use the following Python code:
  import ray
  ray.init(address='ray://100.100.103.186:10001')



In [17]:
cluster.ray_worker_details

{'workers': [{'id': 'gyj9qot42o9fieui',
   'name': 'Ray Worker',
   'project': {'id': 2595,
    'name': 'Ray Distributed ',
    'slug': 'pauldefusco/ray-distributed',
    'html_url': 'https://ml-b74f8940-b97.go01-dem.ylcu-atmi.cloudera.site/pauldefusco/ray-distributed',
    'url': 'https://ml-b74f8940-b97.go01-dem.ylcu-atmi.cloudera.site/api/v1/projects/pauldefusco/ray-distributed',
    'default_engine_type': 'ml_runtime'},
   'owner': {'id': 16,
    'username': 'pauldefusco',
    'name': 'Paul de Fusco',
    'html_url': 'https://ml-b74f8940-b97.go01-dem.ylcu-atmi.cloudera.site/pauldefusco',
    'url': 'https://ml-b74f8940-b97.go01-dem.ylcu-atmi.cloudera.site/api/v1/users/pauldefusco',
    'is_team': False},
   'creator': {'id': 16,
    'username': 'pauldefusco',
    'name': 'Paul de Fusco',
    'html_url': 'https://ml-b74f8940-b97.go01-dem.ylcu-atmi.cloudera.site/pauldefusco',
    'url': 'https://ml-b74f8940-b97.go01-dem.ylcu-atmi.cloudera.site/api/v1/users/pauldefusco'},
   'biller':

In [18]:
runtime_env = {"pip":"-r /home/cdsw/requirements.txt"}

ray.init(address=cluster.get_client_url(),runtime_env=runtime_env)

ConnectionError: ray client connection timeout

In [10]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

train_x, train_y = load_breast_cancer(return_X_y=True)
train_set = RayDMatrix(train_x, train_y)

evals_result = {}
bst = train(
    {
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    train_set,
    evals_result=evals_result,
    evals=[(train_set, "train")],
    verbose_eval=False,
    ray_params=RayParams(num_actors=2, cpus_per_actor=1))

bst.save_model("model.xgb")
print("Final training error: {:.4f}".format(
    evals_result["train"]["error"][-1]))

2023-09-28 17:01:50,619	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
(_wrapped pid=682) [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(_wrapped pid=682) [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=319, ip=100.100.117.56) [17:02:24] task [xgboost.ray]:140091634184448 got new rank 0
(_RemoteRayXGBoostActor pid=322, ip=100.100.175.250) [17:02:24] task [xgboost.ray]:140262712946848 got new rank 1
(_RemoteRayXGBoostActor pid=322, ip=100.100.175.250) FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
(_RemoteRayXGBoostActor pid=322, ip=100.100.175.250) FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
(_RemoteRayXGBoostActor pid=322, ip=100.100.175.250)

Final training error: 0.0053


(_wrapped pid=682) [RayXGBoost] Finished XGBoost training on training data with total N=569 in 20.62 seconds (4.68 pure XGBoost training time).


In [11]:
from xgboost_ray import RayDMatrix, RayParams, predict
from sklearn.datasets import load_breast_cancer
import xgboost as xgb

data, labels = load_breast_cancer(return_X_y=True)

dpred = RayDMatrix(data, labels)

bst = xgb.Booster(model_file="model.xgb")
pred_ray = predict(bst, dpred, ray_params=RayParams(num_actors=2))

print(pred_ray)

(predict pid=682) [RayXGBoost] Created 2 remote actors.


[0.09144595 0.05673993 0.03008196 0.10851309 0.09144595 0.1226117
 0.03008196 0.03145875 0.03795947 0.09715987 0.11452682 0.03008196
 0.03008196 0.04623247 0.06943818 0.03008196 0.03738927 0.03008196
 0.03008196 0.97716796 0.98008084 0.98008084 0.0675434  0.03008196
 0.03008196 0.03620729 0.03974995 0.03008196 0.03008196 0.09527509
 0.03008196 0.03008196 0.03008196 0.03008196 0.03008196 0.03008196
 0.04384386 0.95125544 0.26743954 0.05803299 0.5169782  0.30979052
 0.03008196 0.03145875 0.11796542 0.03620729 0.98008084 0.04324226
 0.98008084 0.93069357 0.9776062  0.9790822  0.9698948  0.03261407
 0.05082424 0.9698948  0.03008196 0.03145875 0.9790822  0.9698948
 0.9731866  0.98008084 0.03008196 0.98008084 0.03145875 0.03145875
 0.9700466  0.9698948  0.86911684 0.9698948  0.03008196 0.98008084
 0.03008196 0.26402172 0.98008084 0.03008196 0.9691353  0.05734929
 0.03958243 0.98008084 0.9700466  0.8453193  0.03008196 0.03008196
 0.98008084 0.03008196 0.14372692 0.03008196 0.9427549  0.922841

(predict pid=682) [RayXGBoost] Starting XGBoost prediction.
(_RemoteRayXGBoostActor pid=379, ip=100.100.175.250) FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
(_RemoteRayXGBoostActor pid=379, ip=100.100.175.250) FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
(_RemoteRayXGBoostActor pid=379, ip=100.100.175.250) FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
(_RemoteRayXGBoostActor pid=379, ip=100.100.175.250) FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
(_RemoteRayXGBoostActor pid=379, ip=100.100.175.250) FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` inste

In [12]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

num_actors = 4
num_cpus_per_actor = 1

ray_params = RayParams(
    num_actors=num_actors, cpus_per_actor=num_cpus_per_actor)

def train_model(config):
    train_x, train_y = load_breast_cancer(return_X_y=True)
    train_set = RayDMatrix(train_x, train_y)

    evals_result = {}
    bst = train(
        params=config,
        dtrain=train_set,
        evals_result=evals_result,
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)
    bst.save_model("model.xgb")

In [13]:
from ray import tune

# Specify the hyperparameter search space.
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

# Make sure to use the `get_tune_resources` method to set the `resources_per_trial`
analysis = tune.run(
    train_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    resources_per_trial=ray_params.get_tune_resources())
print("Best hyperparameters", analysis.best_config)

2023-09-28 17:02:35,105	INFO tune.py:582 -- [output] This uses the legacy output and progress reporter, as Ray client is not supported by the new engine. For more information, see https://github.com/ray-project/ray/issues/36949


(run pid=682) [output] This will use the new output engine with verbosity 2. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949
(run pid=682) AIR_VERBOSITY is set, ignoring passed-in ProgressReporter for now.


(run pid=682) ╭────────────────────────────────────────────────────────────────────╮
(run pid=682) │ Configuration for experiment     train_model_2023-09-28_17-02-36   │
(run pid=682) ├────────────────────────────────────────────────────────────────────┤
(run pid=682) │ Search algorithm                 BasicVariantGenerator             │
(run pid=682) │ Scheduler                        FIFOScheduler                     │
(run pid=682) │ Number of trials                 4                                 │
(run pid=682) ╰────────────────────────────────────────────────────────────────────╯
(run pid=682) 
(run pid=682) View detailed results here: /home/cdsw/ray_results/train_model_2023-09-28_17-02-36
(run pid=682) To visualize your results with TensorBoard, run: `tensorboard --logdir /home/cdsw/ray_results/train_model_2023-09-28_17-02-36`
(run pid=682) 
(run pid=682) Trial status: 4 PENDING
(run pid=682) Current time: 2023-09-28 17:02:45. Total running time: 0s
(run pid=682) Logical resou

(train_model pid=469, ip=100.100.175.250) UserWarning: `ray.tune.integration.xgboost.TuneReportCallback` is deprecated. Use `ray.tune.integration.xgboost.TuneCheckpointReportCallback` instead.
(train_model pid=469, ip=100.100.175.250) [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(train_model pid=469, ip=100.100.175.250) [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=1134) [17:03:01] task [xgboost.ray]:139972998538336 got new rank 0
(_RemoteRayXGBoostActor pid=1137) [17:03:01] task [xgboost.ray]:139818116528160 got new rank 1
(_RemoteRayXGBoostActor pid=364, ip=100.100.117.56) [17:03:01] task [xgboost.ray]:139724534472912 got new rank 2
(_RemoteRayXGBoostActor pid=1137) FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
(_RemoteRayXGBoostActor pid=1137) FutureWarning: is_categorical_dtype is deprecated and will be removed in a future ve

(run pid=682) 
(run pid=682) Trial train_model_d2a6f_00000 finished iteration 1 at 2023-09-28 17:03:06. Total running time: 20s
(run pid=682) ╭──────────────────────────────────────────────────╮
(run pid=682) │ Trial train_model_d2a6f_00000 result             │
(run pid=682) ├──────────────────────────────────────────────────┤
(run pid=682) │ checkpoint_dir_name                              │
(run pid=682) │ time_this_iter_s                         12.4717 │
(run pid=682) │ time_total_s                             12.4717 │
(run pid=682) │ training_iteration                             1 │
(run pid=682) │ train-error                              0.37258 │
(run pid=682) │ train-logloss                            0.65645 │
(run pid=682) ╰──────────────────────────────────────────────────╯
(run pid=682) 
(run pid=682) Trial train_model_d2a6f_00000 finished iteration 2 at 2023-09-28 17:03:06. Total running time: 20s
(run pid=682) ╭──────────────────────────────────────────────────╮
(run pi

...

(train_model pid=469, ip=100.100.175.250) [RayXGBoost] Finished XGBoost training on training data with total N=569 in 12.18 seconds (4.66 pure XGBoost training time).
(train_model pid=469, ip=100.100.175.250) [17:03:18] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
(train_model pid=469, ip=100.100.175.250) [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(train_model pid=469, ip=100.100.175.250) [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=1609) [17:03:25] task [xgboost.ray]:139632089331696 got new rank 0
(_RemoteRayXGBoostActor pid=1683) [17:03:26] task [xgboost.ray]:140048208721952 got new rank 1
(_RemoteRayXGBoostActor pid=583, ip=100.100.117.56) [17:03:26] task [xgboost.ray]:139955944579376 got new rank 2
(_RemoteRayXGBoostActor pid=1683) FutureWarning: is_sparse is

(run pid=682) 
(run pid=682) Trial train_model_d2a6f_00002 finished iteration 1 at 2023-09-28 17:03:31. Total running time: 46s
(run pid=682) ╭──────────────────────────────────────────────────╮
(run pid=682) │ Trial train_model_d2a6f_00002 result             │
(run pid=682) ├──────────────────────────────────────────────────┤
(run pid=682) │ checkpoint_dir_name                              │
(run pid=682) │ time_this_iter_s                         12.6853 │
(run pid=682) │ time_total_s                             12.6853 │
(run pid=682) │ training_iteration                             1 │
(run pid=682) │ train-error                              0.37258 │
(run pid=682) │ train-logloss                            0.59625 │
(run pid=682) ╰──────────────────────────────────────────────────╯
(run pid=682) 
(run pid=682) Trial train_model_d2a6f_00002 finished iteration 2 at 2023-09-28 17:03:31. Total running time: 46s
(run pid=682) ╭──────────────────────────────────────────────────╮
(run pi

(train_model pid=469, ip=100.100.175.250) [RayXGBoost] Finished XGBoost training on training data with total N=569 in 12.77 seconds (5.55 pure XGBoost training time).
(train_model pid=469, ip=100.100.175.250) [17:03:31] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
(train_model pid=469, ip=100.100.175.250) [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(train_model pid=469, ip=100.100.175.250) [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=1864) [17:03:39] task [xgboost.ray]:139806127118416 got new rank 0
(_RemoteRayXGBoostActor pid=1923) [17:03:39] task [xgboost.ray]:140682689087520 got new rank 1
(_RemoteRayXGBoostActor pid=673, ip=100.100.117.56) [17:03:39] task [xgboost.ray]:140333026226528 got new rank 2
(_RemoteRayXGBoostActor pid=1923) FutureWarning: is_sparse is

(run pid=682) 
(run pid=682) Trial train_model_d2a6f_00003 finished iteration 1 at 2023-09-28 17:03:43. Total running time: 58s
(run pid=682) ╭──────────────────────────────────────────────────╮
(run pid=682) │ Trial train_model_d2a6f_00003 result             │
(run pid=682) ├──────────────────────────────────────────────────┤
(run pid=682) │ checkpoint_dir_name                              │
(run pid=682) │ time_this_iter_s                         12.0738 │
(run pid=682) │ time_total_s                             12.0738 │
(run pid=682) │ training_iteration                             1 │
(run pid=682) │ train-error                              0.37258 │
(run pid=682) │ train-logloss                            0.65119 │
(run pid=682) ╰──────────────────────────────────────────────────╯
(run pid=682) 
(run pid=682) Trial train_model_d2a6f_00003 finished iteration 2 at 2023-09-28 17:03:43. Total running time: 58s
(run pid=682) ╭──────────────────────────────────────────────────╮
(run pi

(train_model pid=469, ip=100.100.175.250) [RayXGBoost] Finished XGBoost training on training data with total N=569 in 12.18 seconds (4.66 pure XGBoost training time).
(train_model pid=469, ip=100.100.175.250) [17:03:43] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.


#### The RayParams object is used to configure various settings relating to the distributed training.

In [8]:
ray_params = RayParams(
    num_actors=4,
    gpus_per_actor=1,
)